# REST API
<img src='pics/0.png' />

# Serialization and Deserialization

<img src='pics/chicken-egg.png' />

## serialization

In [ ]:
object = dict(int_list=[1, 2, 3],
 
              text='string',
 
              number=3.44,
 
              boolean=True,
 
              none=None)

print(simple)

In [ ]:
{'none': None, 'int_list': [1, 2, 3], 'text': 'string', 'boolean': True, 'number': 3.44}

In [ ]:
print(json.dumps(object, indent=4))

In [ ]:
{
 
    "text": "string",
 
    "none": null,
 
    "boolean": true,
 
    "number": 3.44,
 
    "int_list": [
 
        1,
 
        2,
 
        3
 
    ]
 
}

### Deserialization

In [ ]:
serialized=json.dumps(object, indent=4)
print(serialized)

In [ ]:
{
    "none": null,
    "int_list": [
        1,
        2,
        3
    ],
    "text": "string",
    "boolean": true,
    "number": 3.44
}

In [ ]:
json.loads(serialized)

In [ ]:
{'text': 'string', 'int_list': [1, 2, 3], 'none': None, 'boolean': True, 'number': 3.44}

# The Serializer class

1. Create a file in the api directory named serializers.py.
3. A serializer class is very similar to a Django Form class, and includes similar validation flags on the various fields, such as required, max_length and default.
4. The create() and update() methods define how fully fledged instances are created or modified when calling serializer.save()


In [ ]:
from rest_framework import serializers
from my_app import models


class AuthorSerializer(serializers.Serializer):
    id = serializers.IntegerField(read_only=True)
    first_name = serializers.CharField(max_length=100)
    last_name = serializers.CharField(max_length=100)
    date_of_birth = serializers.DateField()
    date_of_death = serializers.DateField('Died',)

    def create(self, validated_data):
        """
        Create and return a new `Snippet` instance, given the validated data.
        """
        return models.Author.objects.create(**validated_data)

    def update(self, instance, validated_data):
        """
        Update and return an existing `Snippet`
            instance, given the validated data.
        """
        instance.first_name = validated_data.get(
                                'first_name',
                                instance.first_name
        )

        instance.last_name = validated_data.get(
                                'last_name',
                                instance.last_name
        )

        instance.date_of_birth = validated_data.get(
                                    'date_of_birth',
                                    instance.date_of_birth
        )

        instance.date_of_death = validated_data.get(
                                    'date_of_death',
                                    instance.date_of_death
        )

        instance.save()
        return instance


# Working with Serializers

In [ ]:
python manage.py shell

In [ ]:
>>> from my_app import models
>>> from my_app.api import serializers
>>> from rest_framework.renderers import JSONRenderer

In [ ]:
>>> shakespeare = models.Author(first_name='shakespeare',)
>>> shakespeare.save()
>>> models.Author.objects.all()

<QuerySet [<Author: , shakespeare>]>


In [ ]:
>>> serializers.AuthorSerializer()

AuthorSerializer():
    id = IntegerField(read_only=True)
    first_name = CharField(max_length=100)
    last_name = CharField(max_length=100)
    date_of_birth = DateField()
    date_of_death = DateField('Died')


In [ ]:
>>> serializers.AuthorSerializer(shakespeare)

AuthorSerializer(<Author: , shakespeare>):
    id = IntegerField(read_only=True)
    first_name = CharField(max_length=100)
    last_name = CharField(max_length=100)
    date_of_birth = DateField()
    date_of_death = DateField('Died')


## Python native datatypes

In [ ]:
>>> shakespeare_serialized = serializers.AuthorSerializer(shakespeare)
>>> shakespeare_serialized.data

{'first_name': 'shakespeare', 'last_name': '', 'id': 1, 'date_of_birth': None, 'date_of_death': None}

## rendering Shakespeare into json

In [ ]:
>>> JSONRenderer().render(shakespeare_serialized.data)

b'{"id":2,"first_name":"shakespeare","last_name":"","date_of_birth":null,"date_of_death":null}'

## serialize querysets

In [ ]:
>>> serializer = serializers.AuthorSerializer(models.Author.objects.all(), many=True)
>>> serializer.data

[
    OrderedDict([('id', 3), ('first_name', 'Franz'), ('last_name', 'Kafka'), ('date_of_birth', '2018-03-04'), ('date_of_death', 'Died')])
]



## Using ModelSerializers

### Settings.py

In [ ]:
# Application definition

INSTALLED_APPS = [
    'my_app.apps.MyAppConfig',
    'rest_framework',
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
]

### serailizers.py

In [ ]:
class AuthorSerializer(serializers.ModelSerializer):
    class Meta:
        model = models.Author
        fields = ['id', 'first_name', 'last_name',
                  'date_of_birth', 'date_of_death', ]

### views.py

In [ ]:
from django.http import HttpResponse

from rest_framework import generics

from . import serializers
from my_app import models


def test(request, ):
    return HttpResponse('well done!')


class AuthorListApiView(generics.ListAPIView):
    serializer_class = serializers.AuthorSerializer

    def get_queryset(self, ):
        return models.Author.objects.all()


### url.py

In [ ]:
from django.conf.urls import url

from . import views


urlpatterns = [
    url(r'^test/$', view=views.test, name='test'),
    url(
        r'^author/$',
        view=views.AuthorListApiView.as_view(),
        name='author-list-api-view'
    ),

]

### output

<img src='pics/Screenshot from 2018-03-06 15-21-21.png' />

### settings.py

In [ ]:
INSTALLED_APPS = [
    'my_app.apps.MyAppConfig',
    'rest_framework',
    'rest_framework_swagger',
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
]


### urls.py

In [ ]:
from django.conf.urls import url

from . import views

from rest_framework_swagger.views import get_swagger_view

swagger = get_swagger_view(title='Public API')

urlpatterns = [
    url(r'^test/$', view=views.test, name='test'),
    url(
        r'^author/$',
        view=views.AuthorListApiView.as_view(),
        name='author-list-api-view'
    ),
    url(r'^swagger/$', view=swagger, name='swagger'),
]


<img src='pics/Screenshot from 2018-03-06 15-46-35.png' />